In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import json

font = {'size' : 8}
mpl.rc('font', **font)

In [ ]:
def get_filename(num_vars, canalyzing_depth):
    return "./data/num_vars=" + str(num_vars) + "_depth=" + str(canalyzing_depth) + ".json"

# The number and average size of attractors vs. depth (Fig 1)

We plot how the empitical mean of the number of attractors and the average attractor size depends on the canalizing depth in the case $n = 15$.

There are 50000 samples for each depth.

In [ ]:
def get_statistics(function, num_vars):
    mean = []
    stddev = []
    for can_depth in range(num_vars + 1):
        with open(get_filename(num_vars, can_depth), "r") as file:
            data = json.load(file)
            #print("Num samples %d" % len(data))
            values = np.fromiter(map(function, data), float)
        mean.append(np.mean(values))
        stddev.append(np.std(values))
    return {"means" : mean, "stddev": stddev}

In [ ]:
def num_attractors(dds):
    return len(dds) * 1.

In [ ]:
def avg_attractor_size(dds):
    return sum([att[0] for att in dds]) * 1. / len(dds)

In [ ]:
funcs = {"# of attractors" : num_attractors, "Avg attractor size" : avg_attractor_size}

In [ ]:
num_vars = 15
stats = dict()
for name, f in funcs.items():
    stats[name] = get_statistics(f, num_vars)

In [ ]:
yticks = {"# of attractors" : list(range(9)), "Avg attractor size" : list(range(0, 70, 10))}
filename = {"# of attractors" : "N_mean.pdf", "Avg attractor size" : "AS_mean.pdf"}
for name, st in stats.items():
    plt.figure(figsize = (3, 2), dpi = 150)
    plt.xlabel("Canalizing depth")
    plt.ylabel(name)
    plt.yticks(yticks[name])
    plt.xticks(range(0, num_vars + 1, 2))
    plt.minorticks_on()
    height = max(st["means"]) + max(st["stddev"])
    (_, caps, _) = plt.errorbar([_ for _ in range(num_vars + 1)], st["means"], st["stddev"], 
             ecolor = 'black', color = 'g', errorevery = 2, marker = 'o', 
             markersize = 3, elinewidth = 0.5, capsize = 3)
    for cap in caps:
        cap.set_color('black')
        cap.set_markeredgewidth(0.5)
    plt.savefig(filename[name], bbox_inches = "tight")

# Distribution of the number and average size of attractors (Fig 2)
We plot these distributions of the numer of variables $n = 12$ and canalizing depths 0, 1, 3, and 12.

In [ ]:
def get_distribution(function, num_vars, can_depth):
    with open(get_filename(num_vars, can_depth), "r") as file:
        data = json.load(file)
        values = np.fromiter(map(function, data), float)
        dist = np.histogram(values, range(int(max(values))), density = True)
    return (dist[1][:-1], dist[0])

In [ ]:
num_vars = 12
depths = [0, 1, 3, 12]
dists_all = dict()
max_x = dict()
for name, f in funcs.items():
    dists_all[name] = dict()
    for d in depths:
        dists_all[name][d] = get_distribution(f, num_vars, d)
    max_x[name] = min([len(dist[0]) for dist in dists_all[name].values()]) + 1

In [ ]:
yticks_step = {"# of attractors" : 2, "Avg attractor size" : 5}
filename = {"# of attractors" : "N_freq.pdf", "Avg attractor size" : "AS_freq.pdf"}
for name, dists in dists_all.items():
    plt.figure(figsize = (3, 2), dpi = 150)
    plt.xlabel(name)
    plt.ylabel("Frequency")
    plt.xticks(range(0, max_x[name], yticks_step[name]))
    plt.yticks([0.05 * i for i in range(0, 6)])
    plt.minorticks_on()
    for d, dist in dists.items():
        plt.plot(dist[0][:max_x[name]], dist[1][:max_x[name]])

    plt.legend(["k = %d" % d for d in depths])
    plt.savefig(filename[name], bbox_inches = "tight")

# Relative decreases (Fig 3)

In [ ]:
def compute_relative_decrease(function, num_vars, depth):
    values = []
    for d in (0, depth):
        with open(get_filename(num_vars, d), "r") as file:
            data = json.load(file)
            values.append(np.mean(np.fromiter(map(function, data), float)))
    return values[1] / values[0]

In [ ]:
nums_vars = list(range(4, 21))
decreases = dict()
depths = [1, 2, 3]
for name, f in funcs.items():
    decreases[name] = dict()
    for d in depths:
        decreases[name][d] = [compute_relative_decrease(f, n, d) for n in nums_vars]
    decreases[name]["n"] = [compute_relative_decrease(f, n, n) for n in nums_vars]

In [ ]:
legend = {
    "# of attractors" : [r'$N_1(n)$', r'$N_2(n)$', r'$N_3(n)$', r'$N_n(n)$'], 
    "Avg attractor size" : [r'$\mathrm{AS}_1(n)$', r'$\mathrm{AS}_2(n)$', r'$\mathrm{AS}_3(n)$', r'$\mathrm{AS}_n(n)$']
}
filename = {"# of attractors" : "N_ratio.pdf", "Avg attractor size" : "AS_ratio.pdf"}
for name, data in decreases.items():
    plt.figure(figsize = (3, 2.2), dpi = 150)
    plt.xlabel("# of variables")
    plt.ylabel("Relative decrease")
    plt.ylim(0, 0.9)
    plt.yticks([0.2 * i for i in range(5)])
    plt.xticks(range(4, 22, 2))
    plt.minorticks_on()
    plt.plot(nums_vars, data[1])
    plt.plot(nums_vars, data[2])
    plt.plot(nums_vars, data[3])
    plt.plot(nums_vars, data["n"])
    plt.legend(legend[name])
    plt.savefig(filename[name], bbox_inches = "tight")